In [2]:
import tensorflow as tf

import keras

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os

# image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from os import listdir

# CNN
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D  
from tensorflow.keras.layers import MaxPooling2D 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout  

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array  
from tensorflow.keras import Sequential

from PIL import Image

import os

import warnings
warnings.filterwarnings('ignore')


In [3]:
# pretrained models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

# Creating dirs to store data

In [4]:
data_path = 'Damage Classification/full'
train_path = 'Damage Classification/train'
val_path = 'Damage Classification/val'

In [5]:
os.listdir(data_path)

['01-minor', '02-moderate', '03-severe']

In [16]:
for i in os.listdir(data_path):
    new_dir=os.path.join(train_path,i)
    os.makedirs(new_dir)
    print("Created : ",new_dir)
    new_dir=os.path.join(val_path,i)
    os.makedirs(new_dir)
    print("Created : ",new_dir)

Created :  Damage Classification/train\01-minor
Created :  Damage Classification/val\01-minor
Created :  Damage Classification/train\02-moderate
Created :  Damage Classification/val\02-moderate
Created :  Damage Classification/train\03-severe
Created :  Damage Classification/val\03-severe


# Data Augmentation

In [17]:
def augment_data(data_dir, output_dir, num_images_per_sample, save_prefix='augmented', save_format='jpeg'):

  image_paths = [os.path.join(data_dir, f) for f in listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]

  os.makedirs(output_dir, exist_ok=True)
  train_datagen = ImageDataGenerator(
    rescale=1.0/255,                # Rescale pixel values to [0, 1]
    rotation_range=20,              # Random rotation within 20 degrees
    width_shift_range=0.2,          # Random horizontal shift by 20% of image width
    height_shift_range=0.2,         # Random vertical shift by 20% of image height
    horizontal_flip=True,           # Random horizontal flipping
    fill_mode='nearest'             # Fill mode for new pixels after shifts/rotations
    )      

  
  images = []
  for image_path in image_paths:
    img = load_img(image_path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)  
    images.append(x)

  
  for i, image in enumerate(images):
    batch = train_datagen.flow(image, batch_size=32)
    for j in range(num_images_per_sample):
      augmented_img = next(batch)[0]  
      augmented_img = array_to_img(augmented_img)  
      filename = f"{save_prefix}_{i}_{j}.{save_format}"
      augmented_img.save(os.path.join(output_dir, filename))  


In [24]:
for i in os.listdir(data_path):
    data_dir = os.path.join(data_path,i)
    print(data_dir)
    output_dir=os.path.join(train_path,i)
    print(output_dir)
    augment_data(data_dir,output_dir=output_dir, num_images_per_sample=15)
    print(i," done")

Damage Classification/full\01-minor
Damage Classification/train\01-minor
01-minor  done
Damage Classification/full\02-moderate
Damage Classification/train\02-moderate
02-moderate  done
Damage Classification/full\03-severe
Damage Classification/train\03-severe
03-severe  done


# Resizing Images

In [25]:
def resize_images(folder_path, output_folder=None):

  for filename in os.listdir(folder_path):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
      filepath = os.path.join(folder_path, filename)
      img = Image.open(filepath)
      img = img.resize((256, 256))  # Resize without antialiasing (if not using Pillow 5.1.0+)

      if output_folder:
        new_path = os.path.join(output_folder, filename)
        img.save(new_path)
      else:
        img.save(filepath)


In [30]:
for i in os.listdir(train_path):
    path=os.path.join(train_path,i)
    resize_images(path)
    print("Done : ",path)

Done :  Damage Classification/train\01-minor
Done :  Damage Classification/train\02-moderate
Done :  Damage Classification/train\03-severe


In [60]:
for i in os.listdir(val_path):
    path=os.path.join(val_path,i)
    resize_images(path)
    print("Done : ",path)

Done :  Damage Classification/val\01-minor
Done :  Damage Classification/val\02-moderate
Done :  Damage Classification/val\03-severe


# Generating Data

In [6]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_path,
    color_mode='rgb',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    # save_to_dir='final_data/train_data'
    )
class_names = train_generator.class_indices

# Print class names and their corresponding indices
for class_name, index in class_names.items():
  print(f"Class Name: {class_name}, Index: {index}")

Found 14685 images belonging to 3 classes.
Class Name: 01-minor, Index: 0
Class Name: 02-moderate, Index: 1
Class Name: 03-severe, Index: 2


In [7]:
datagen = ImageDataGenerator(rescale=1./255)
val_generator = datagen.flow_from_directory(
    val_path,
    color_mode='rgb',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    # save_to_dir='final_data/train_data'
    )
class_names = train_generator.class_indices

# Print class names and their corresponding indices
for class_name, index in class_names.items():
  print(f"Class Name: {class_name}, Index: {index}")

Found 979 images belonging to 3 classes.
Class Name: 01-minor, Index: 0
Class Name: 02-moderate, Index: 1
Class Name: 03-severe, Index: 2


# Creating The Model

### Callbacks

In [8]:
earlystop = EarlyStopping(patience=20)

In [9]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [10]:
callbacks = [earlystop, learning_rate_reduction]
epochs=7

### Model

In [11]:
# # create CNN model

# model = Sequential()

# model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

# model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

# model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
# model.add(Flatten())

# model.add(Dense(512,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))

# model.add(Dense(64,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Dense(2,activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# model.summary()

## VGG16 MODEL

In [14]:
input_img = Input(shape=(256, 256, 3))

vgg16_base = VGG16(weights='imagenet', include_top=False, input_tensor=input_img)

vgg16_base.trainable = True 

x = Flatten()(vgg16_base.output)

x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(3, activation='softmax')(x)  

model = Model(inputs=input_img, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 256, 256, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 256, 256, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 128, 128, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 128, 128, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 64, 64, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 32, 32, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 16, 16, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 8, 8, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │      16,777,728 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 31,657,027 (120.76 MB)

 Trainable params: 31,657,027 (120.76 MB)

 Non-trainable params: 0 (0.00 B)

In [66]:
history = model.fit(
    train_generator, 
    epochs=10,
    validation_data=val_generator,
    # validation_batch_size=0.2,
    callbacks=callbacks
)

Epoch 1/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 4538s 10s/step - accuracy: 0.4859 - loss: 1.9495 - val_accuracy: 0.6701 - val_loss: 0.7413 - learning_rate: 0.0010
Epoch 2/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1861s 4s/step - accuracy: 0.6292 - loss: 0.7955 - val_accuracy: 0.7283 - val_loss: 0.6187 - learning_rate: 0.0010
Epoch 3/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1389s 3s/step - accuracy: 0.6774 - loss: 0.7202 - val_accuracy: 0.7232 - val_loss: 0.6044 - learning_rate: 0.0010
Epoch 4/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1324s 3s/step - accuracy: 0.7139 - loss: 0.6375 - val_accuracy: 0.7835 - val_loss: 0.5192 - learning_rate: 0.0010
Epoch 5/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1267s 3s/step - accuracy: 0.7399 - loss: 0.5855 - val_accuracy: 0.7937 - val_loss: 0.4930 - learning_rate: 0.0010
Epoch 6/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1167s 3s/step - accuracy: 0.7610 - loss: 0.5529 - val_accuracy: 0.8182 - val_loss: 0.4588 - learning_rate: 0.0010
Epoch 7/10
459/459 ━━━━━━━━━━━━━━━━━━━━ 1088s 2s/step - accuracy: 0.7821 - 

In [1]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_image(img_path, model):

    img = load_img(img_path, target_size=(256, 256))
    
    img_array = img_to_array(img)
    
    img_array = img_array / 255.0
    
    img_array = img_array[None, ...] 
    
    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5 and prediction[0][0] > prediction[0][1]:
        return 0
    else: 
        return 1


img_path = "Training/00-front/0004.JPEG"

prediction = predict_image(img_path, model)

print(prediction)


NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import Model

model.save('damage_classification_model.h5') 

In [ ]:
new_model = model.load_model('damage_classification_model.h5')

In [ ]:
pred=predict_image(img_path, new_model)
pred